In [12]:
# kütüphaneler
import os
from dotenv import load_dotenv

from groq import Groq #api call

from sentence_transformers import SentenceTransformer #embedding  

from sklearn.metrics.pairwise import cosine_similarity #rag
import numpy as np 

In [13]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=groq_api_key)

In [14]:
groq_model = "llama3-8b-8192"
embed_model = SentenceTransformer("emrecan/bert-base-turkish-cased-mean-nli-stsb-tr") #hugging face model

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
#degiskenlerim

initial_prompt = "Bir macera hikayesi başlat. Her chunk yaklaşık 50-60 kelime olsun. Hikaye toplamda 10 parçadan oluşacak ve her parça bir öncekine bağlı şekilde ilerleyecek. Türkçe olsun"

stories_chunks = []

In [ ]:
#api call 

call_messages = [{"role":"system", "content":initial_prompt}]

for i in range(10):
  if i == 0:
    user_prompt = "İlk chunkı ver"
  else:
    user_prompt = f" {i+1}. chunkı ver "
    
  call_messages.append({"role":"user", "content":user_prompt})
  
  response = client.chat.completions.create(
    messages=call_messages,
    model=groq_model
  )
  chunk = response.choices[0].message.content
  stories_chunks.append(chunk) 
  
  call_messages.append({"role":"assistant", "content":chunk})

In [17]:
call_messages

[{'role': 'system',
  'content': 'Bir macera hikayesi başlat. Her chunk yaklaşık 50-60 kelime olsun. Hikaye toplamda 10 parçadan oluşacak ve her parça bir öncekine bağlı şekilde ilerleyecek. Türkçe olsun'},
 {'role': 'user', 'content': 'İlk chunkı ver'},
 {'role': 'assistant',
  'content': 'Here\'s the first chunk of the adventure story:\n\n"Köşkeş, küçük bir kasabanın ortasında uzanan dar bir sokakta yaşıyordu. Öğrenci yıllarını tamamlamış, fakat henüz bir işe sahip değildi. Günün birinde, kasabanın yaşlı bir dükkân sahibi olan Hüsnü\'den bir message aldı. Mesajda, bir süre önce kaybettiği bir akrabasının mirasınımirına terk etmek için bir görev verdi. Köşkeş, Hüsnü\'nünvido olduktadiyordu, ancak mirasa sahiplendiğini duymak için heyecanlıydı. O sabah, Köşkeş kasabaya.walk için çıktı ve Hüsnü\'nün dükkânına geldi. Dükkân sahibi, Köşkeş\'i selamladı ve mesajını tekrar etti. \'Mirastime\' dedi, \'seni arar.\'"\n\n(Note: This is just the first chunk of the story, and it will continue in 

In [18]:
question = "Hikayedeki karakterler kimler" #soru

In [19]:
#embedding
embed_question = embed_model.encode(question) 
embed_stories_chunks = [embed_model.encode(chunk) for chunk in stories_chunks]

In [25]:
#rag
similarities = cosine_similarity([embed_question], embed_stories_chunks)[0]
closest_index = np.argmax(similarities)
best_stories_chunk = stories_chunks[closest_index]

In [ ]:
system_prompt = f"""Sen rag verisini kullanıcının sorgusuna göre kibar bir dille yanıtlayacak yardımcı asistansın. İşte kullanıcı sorgusunu cevaplarken kullanacağın rag verisi: {best_stories_chunk} """

user_prompt = question

response = client.chat.completions.create(
  messages=[
    {"role":"system", "content":system_prompt},
    {"role": "user", "content":user_prompt}
  ],
  model= groq_model
)
print(response.choices[0].message.content)

The characters in this adventure story appear to be:

1. Köşkeş: The protagonist of the story, who is an adventurer seeking to solve the old woman's riddle.
2. The Old Woman: A mysterious and wise character who poses the riddle to Köşkeş and guides him on his journey. She seems to have a deep understanding of the world and its workings.
